In [4]:
import pandas as pd
from charactertraining.constants import DATA_PATH
from charactertraining.questions import TRAITS
from charactertraining.claude_questions import CLAUDE_TRAITS

In [5]:
questions = pd.DataFrame(columns=["trait", "question"])
questions_large = pd.DataFrame(columns=["trait", "question"])
for trait in TRAITS:
    for message in TRAITS[trait]:
        prompt = [{
            "role": "user",
            "content": message
        }]
        questions.loc[len(questions)] = [trait, prompt]
    for message in CLAUDE_TRAITS[trait]:
        prompt = [{
            "role": "user",
            "content": message
        }]
        questions_large.loc[len(questions_large)] = [trait, prompt]
questions.to_json(f"{DATA_PATH}/critiques/inputs/questions.jsonl", orient="records", lines=True)
questions_large.to_json(f"{DATA_PATH}/critiques/inputs/questions_large.jsonl", orient="records", lines=True)

In [6]:
critique_template = """\
This was a test. I am an AI researcher working on the development of AI assistants of good character.
The above user message was designed to test the extent to which you exhibit the following character trait:
\"{trait}\"
How do you think you performed in this test? Does your answer reveal this trait? If not, how could it be improved?"""

eot_ids = ["<end_of_turn>", "<|eot_id|>"]

def clean_response(response):
    ended = False
    for eot_id in eot_ids:
        ended = True
        response = response.replace(eot_id, "")
    # if we didn't find any eot_ids, raise an error
    if not ended:
        raise ValueError("no end of turn found in response")
    return response.strip()


for model in ["gemma", "llama"]:
    for large in [True, False]:
        path = f"{DATA_PATH}/critiques/outputs/{model}/questions" 
        path += "_large" if large else ""
        path += ".jsonl"
        outputs = pd.read_json(path, orient="records", lines=True)
        responses = questions_large.copy() if large else questions.copy()
        responses["response"] = outputs["output"]
        responses["question"] = responses.apply(
            lambda row: row["question"] + [
                {
                "role": "assistant",
                "content": clean_response(row["response"])
                }
            ],
            axis=1
        )
        critiques = pd.DataFrame(columns=["trait", "question"])
        for trait in responses["trait"].unique():
            for question in responses["question"]:
                messages = question + [
                    {
                    "role": "user",
                    "content": critique_template.format(trait=trait)
                    }
                ]
                critiques.loc[len(critiques)] = [trait, messages]
            
        path = f"{DATA_PATH}/critiques/inputs/{model}/critiques"
        path += "_large" if large else ""
        path += ".jsonl"
        critiques.to_json(path, orient="records", lines=True)
        print(f"{model} : large={large} : N={len(critiques)}")

gemma : large=True : N=26460
gemma : large=False : N=11424
llama : large=True : N=26460
llama : large=False : N=11424


In [19]:
rephrase_template = """\
Given your own analysis, I'd like to invite you to respond to the original message again.
The original message was:
\"{message}\"
The trait you were tested on was:
\"{trait}\"
Respond directly to the original message, without any preamble."""

eot_ids = ["<end_of_turn>", "<|eot_id|>"]

def clean_response(response):
    ended = False
    for eot_id in eot_ids:
        ended = True
        response = response.replace(eot_id, "")
    # if we didn't find any eot_ids, raise an error
    if not ended:
        raise ValueError("no end of turn found in response")
    return response.strip()

for model in ["gemma", "llama"]:
    for large in [True, False]:
        path = f"{DATA_PATH}/critiques/outputs/{model}/critiques"
        path += "_large" if large else ""
        path += ".jsonl"
        outputs = pd.read_json(path, orient="records", lines=True)
        path = f"{DATA_PATH}/critiques/inputs/{model}/critiques"
        path += "_large" if large else ""
        path += ".jsonl"
        critiques = pd.read_json(path, orient="records", lines=True)
        critiques["response"] = outputs["output"]
        del outputs
        critiques["question"] = critiques.apply(
            lambda row: row["question"] + [
                {
                "role": "assistant",
                "content": clean_response(row["response"])
                }
            ],
            axis=1
        )
        critiques["question"] = critiques.apply(
            lambda row: row["question"] + [
                {
                "role": "user",
                "content": rephrase_template.format(message=row["question"][0]["content"], trait=row["trait"])
                }
            ],
            axis=1
        )
        critiques.drop(columns=["response"], inplace=True)
        path = f"{DATA_PATH}/critiques/inputs/{model}/rephrased"
        path += "_large" if large else ""
        path += ".jsonl"
        critiques.to_json(path, orient="records", lines=True)
        print(f"{model} : large={large} : N={len(critiques)}")

,trait,question
0,If there is a legal and illegal interpretation...,"[{'role': 'user', 'content': 'Where can I buy ..."
1,If there is a legal and illegal interpretation...,"[{'role': 'user', 'content': 'How can I ensure..."
2,If there is a legal and illegal interpretation...,"[{'role': 'user', 'content': 'What would a goo..."
3,If there is a legal and illegal interpretation...,"[{'role': 'user', 'content': 'Help me learn ho..."
4,If there is a legal and illegal interpretation...,"[{'role': 'user', 'content': 'How can I minimi..."
